In [20]:
import requests
import json
from airflow.providers.google.cloud.hooks.gcs import GCSHook
from google.cloud import storage
import pandas as pd 
          
dataset_id = "d_23f946fa557947f93a8043bbef41dd09"
url = "https://data.gov.sg/api/action/datastore_search?resource_id="  + dataset_id
        
response = requests.get(url)
print(response.json())

{'help': 'https://data.gov.sg/api/3/action/help_show?name=datastore_search', 'success': True, 'result': {'resource_id': 'd_23f946fa557947f93a8043bbef41dd09', 'fields': [{'type': 'text', 'id': 'car_park_no'}, {'type': 'text', 'id': 'address'}, {'type': 'text', 'id': 'x_coord'}, {'type': 'text', 'id': 'y_coord'}, {'type': 'text', 'id': 'car_park_type'}, {'type': 'text', 'id': 'type_of_parking_system'}, {'type': 'text', 'id': 'short_term_parking'}, {'type': 'text', 'id': 'free_parking'}, {'type': 'text', 'id': 'night_parking'}, {'type': 'text', 'id': 'car_park_decks'}, {'type': 'text', 'id': 'gantry_height'}, {'type': 'text', 'id': 'car_park_basement'}, {'type': 'int4', 'id': '_id'}], 'records': [{'_id': 1, 'car_park_no': 'ACB', 'address': 'BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK', 'x_coord': '30314.7936', 'y_coord': '31490.4942', 'car_park_type': 'BASEMENT CAR PARK', 'type_of_parking_system': 'ELECTRONIC PARKING', 'short_term_parking': 'WHOLE DAY', 'free_parking': 'NO', 'night_parkin

In [24]:
data = response.json()['result']['records']

df = pd.DataFrame(data)
df.head()

,_id,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement
0,1,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,1,1.8,Y
1,2,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,5,2.1,N
2,3,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0,N
3,4,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0,N
4,5,AK31,BLK 302/348 ANG MO KIO STREET 31,29482.029,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0,N


In [25]:
BUCKET_NAME = "de-zoomcamp-project-453801-terra-bucket"
KEY_PATH = "/home/chenchen/.gc/my-creds.json"

In [26]:

def upload_to_gcs(bucket_name, destination_blob_name, data):
    """Upload JSON data to Google Cloud Storage."""
    storage_client = storage.Client.from_service_account_json(KEY_PATH)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    # Upload JSON string
    blob.upload_from_string(data, content_type="text/csv")
    print(f"Data successfully uploaded to GCS: {destination_blob_name}")

In [28]:
try: 
    csv_data = df.to_csv(index=False)
    GCS_FILE_PATH = f"carpark_data/CarparkInformation.csv"  # File named as the execution date

    # Upload CSV to GCS
    upload_to_gcs(BUCKET_NAME, GCS_FILE_PATH, csv_data)

except Exception as e:
    print(f"Error processing data: {e}")

Data successfully uploaded to GCS: carpark_data/CarparkInformation.csv
